# molecular descriptors for ALDH1 inhibitors:

Different descriptors can be determined on the given molecules to find a (causal) relation between the molecule and the ability to inhibit ALDH1. 

Molecular descriptors are for instance: 
-   molecular mass
-   nr carbon atoms
-   nr hydrogen atoms  
-   nr of bonds
-   nr of branches
-   nr double bindings
-   nr triple bindings
-   cyclic structures
-   Aromaticity (indicated by lower letters)
    -   aromatic nitrogen
-   (tetra hedral) chirality
- nr of rings (e.g. cubane)

### rdkit has automatic implemented descriptors and Fingerprints:

This is used now for the generation of descriptors. also a couple of fingerprint variables can be included. 


In [1]:
import numpy as np
import pandas as pd
from tkinter import filedialog as fd
from rdkit import Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, Descriptors, AllChem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
IPythonConsole.ipython_useSVG=True



In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
import joblib

In [3]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule
    
        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [4]:
filename = fd.askopenfilename()
AHDL1Inhibitors = pd.read_csv(filename ,header = None)

2023-06-01 09:25:04.273 python[85859:8735370] +[CATransaction synchronize] called within transaction
2023-06-01 09:25:04.542 python[85859:8735370] +[CATransaction synchronize] called within transaction


In [5]:
allTestedMolecules = AHDL1Inhibitors[0] # firts 3 for testing, needs to change for all molecules (remove[0:4])
MolList = allTestedMolecules.values.tolist()
with open('AllTestedMols.txt', 'w') as fp:
    for mol in MolList:
        # write each item on a new line
        fp.write("%s\n" % mol)
    print('Done')

Done


In [6]:
suppl = Chem.SmilesMolSupplier('AllTestedMols.txt')
mols = [m for m in suppl]
# len(mols)


[09:25:20] WARNING: no name column found on line 1
[09:25:20] WARNING: no name column found on line 2
[09:25:20] WARNING: no name column found on line 3
[09:25:20] WARNING: no name column found on line 4
[09:25:20] WARNING: no name column found on line 5
[09:25:20] WARNING: no name column found on line 6
[09:25:20] WARNING: no name column found on line 7
[09:25:20] WARNING: no name column found on line 8
[09:25:20] WARNING: no name column found on line 9
[09:25:20] WARNING: no name column found on line 10
[09:25:20] WARNING: no name column found on line 11
[09:25:20] WARNING: no name column found on line 12
[09:25:20] WARNING: no name column found on line 13
[09:25:20] WARNING: no name column found on line 14
[09:25:20] WARNING: no name column found on line 15
[09:25:20] WARNING: no name column found on line 16
[09:25:20] WARNING: no name column found on line 17
[09:25:20] WARNING: no name column found on line 18
[09:25:20] WARNING: no name column found on line 19
[09:25:20] WARNING: n

In [7]:
allDescrs = [getMolDescriptors(m) for m in mols]
allDescrsDf = pd.DataFrame(allDescrs)
allDescrsDf.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.083531,13.083531,0.001173,-0.683140,0.520365,463.542,434.310,463.233188,178,0,...,0,0,0,0,1,0,0,0,0,0
1,12.170097,12.170097,0.066966,-0.066966,0.498564,378.457,360.313,378.115047,136,0,...,1,0,0,0,0,0,0,0,0,0
2,10.905837,10.905837,0.016881,-0.016881,0.382043,477.589,444.325,477.260865,184,0,...,0,0,0,0,1,0,0,0,0,0
3,11.562446,11.562446,0.270607,-0.454447,0.795948,330.609,317.505,328.981818,96,0,...,0,0,0,0,0,0,0,0,0,0
4,12.108866,12.108866,0.086947,-3.251317,0.687618,419.553,402.417,419.043204,140,0,...,0,1,0,0,0,1,0,1,0,0


### Generate Fingerprints

In [8]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [9]:
fingerp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]
x = rdkit_numpy_convert(fingerp)
x.shape

(1000, 2048)

In [10]:
fpgen = AllChem.GetRDKitFPGenerator()
fps = [fpgen.GetFingerprint(x) for x in mols]

In [11]:
fpgen = AllChem.GetMorganGenerator(radius=2)
fp1 = fpgen.GetSparseCountFingerprint(mols[1])
# check imbalance dataset:
y = AHDL1Inhibitors[1][1:].astype(int)
sum(y)/len(y)
# mild imbalance

0.3

Use data for Machine Learning: 
- Random forest
- SVM

In [12]:
# split data in train and test sets. Set the testset size to 20%
seed = 13
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.20, random_state=seed)
# create folds for cross- validation
cv = StratifiedKFold(n_splits=5, shuffle=True,random_state=seed)

In [16]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(xTrain, yTrain)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   2   3   4   6   8   9  10  11  12  13  14  15  16  17  19  20
  22  23  24  25  26  27  29  30  31  32  33  34  35  36  38  39  40  41
  42  43  44  45  46  48  50  51  52  53  54  55  56  58  59  60  61  62
  63  64  65  66  68  70  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  94  95  96  97  99 101 102 103 104 105
 106 107 108 109 110 111 115 116 117 118 119 120 122 123 125 126 129 131
 134 135 136 137 138 139 140 141 142 143 144 145 146 148 149 150 151 153
 154 155 156 157 158 159 160 161 162 163 165 166 167 168 169 171 172 173
 174 176 177 178 179 181 182 183 185 186 187 189 190 191 192 194 196 197
 198 199 200 201 202 203 205 206 207 208 210 211 212 213 214 215 216 218
 219 221 222 223 224 225 226 228 229 231 233 234 236 237 238 239 240 243
 244 245 247 248 251 252 253 254 255 256 258 259 260 261 262 263 264 265
 269 272 273 274 276 278 279 281 282 283 285 288 289 290 291 292 295 297
 299 300 301 303 304 305 306 307 308

In [13]:
# Scale inputs
scale = StandardScaler().fit(xTrain)
xTrain = scale.transform(xTrain)


In [18]:
# save data for future use
joblib.dump(scale, "Fingerprints.pkl", compress=3)

['Fingerprints.pkl']

In [19]:
# create grid search dictionary
param_grid = {"max_features": [xTrain.shape[1] // 10, xTrain.shape[1] // 7, 
                               xTrain.shape[1] // 5, xTrain.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [20]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [21]:
# run model building
m.fit(xTrain, yTrain)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [204, 292, 409, 682],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [22]:
m.best_params_

{'max_features': 204, 'n_estimators': 500}

In [23]:
m.best_score_

0.71125

In [24]:
m.cv_results_


{'mean_fit_time': array([ 1.10813403,  2.69689684,  5.35178924,  1.45882792,  3.68526316,
         7.40756922,  1.91291466,  4.89108715,  9.79830856,  3.14473596,
         7.30430646, 14.3517221 ]),
 'std_fit_time': array([0.05542844, 0.07978297, 0.1019278 , 0.05778478, 0.10112704,
        0.24922392, 0.05683592, 0.16722132, 0.3569647 , 0.12766323,
        0.34373597, 0.84235497]),
 'mean_score_time': array([0.0116549 , 0.02610278, 0.05079942, 0.01149559, 0.02667508,
        0.05284624, 0.01140223, 0.02660689, 0.05767837, 0.01129394,
        0.02549133, 0.0500567 ]),
 'std_score_time': array([0.000487  , 0.00088111, 0.00115236, 0.0006131 , 0.00078685,
        0.00038946, 0.00099754, 0.0008222 , 0.01192052, 0.00041175,
        0.00079633, 0.0017285 ]),
 'param_max_features': masked_array(data=[204, 204, 204, 292, 292, 292, 409, 409, 409, 682, 682,
                    682],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False

In [25]:
joblib.dump(m, "RFmodelMorganFingerprint.pkl", compress=3)

['RFmodelMorganFingerprint.pkl']

Load model (also to check if it works)

In [29]:
scale = joblib.load("Fingerprints.pkl")
# scale descriptors of the test set compounds
xTest = scale.transform(xTest)
# predict logBB class
predRF = m.predict(xTest)
predRF

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0])

In [30]:
accuracy_score(yTest, predRF)

0.745

In [32]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(xTest)
pred_prob

array([[0.592, 0.408],
       [0.678, 0.322],
       [0.806, 0.194],
       [0.754, 0.246],
       [0.78 , 0.22 ],
       [0.46 , 0.54 ],
       [0.828, 0.172],
       [0.738, 0.262],
       [0.906, 0.094],
       [0.78 , 0.22 ],
       [0.634, 0.366],
       [0.844, 0.156],
       [0.772, 0.228],
       [0.43 , 0.57 ],
       [0.442, 0.558],
       [0.56 , 0.44 ],
       [0.762, 0.238],
       [0.646, 0.354],
       [0.794, 0.206],
       [0.73 , 0.27 ],
       [0.688, 0.312],
       [0.93 , 0.07 ],
       [0.744, 0.256],
       [0.442, 0.558],
       [0.722, 0.278],
       [0.914, 0.086],
       [0.642, 0.358],
       [0.67 , 0.33 ],
       [0.856, 0.144],
       [0.638, 0.362],
       [0.82 , 0.18 ],
       [0.718, 0.282],
       [0.672, 0.328],
       [0.93 , 0.07 ],
       [0.538, 0.462],
       [0.874, 0.126],
       [0.86 , 0.14 ],
       [0.718, 0.282],
       [0.868, 0.132],
       [0.618, 0.382],
       [0.526, 0.474],
       [0.77 , 0.23 ],
       [0.368, 0.632],
       [0.9

In [26]:
# setup threshold
threshold = 0.8
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold
da

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [34]:
# calc statistics
accuracy_score(np.asarray(yTest)[da], predRF[da])

0.8823529411764706

In [35]:
# calc coverage
sum(da) / len(da)

0.34

SVM approach

In [15]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [16]:
# run model building
svm.fit(xTrain, yTrain)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [17]:
svm.best_params_

{'C': 1, 'gamma': 1e-06}

In [18]:
svm.best_score_

0.69375

In [19]:
# save model
joblib.dump(svm, "SVMmorganFingerprint.pkl", compress=3)

['SVMmorganFingerprint.pkl']

In [20]:
# predict logBB for the test set compounds
pred_svm = svm.predict(xTest)
pred_svm


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [22]:
# calc statistics
print("Accuracy = ", accuracy_score(yTest, pred_svm))
print("MCC = ", matthews_corrcoef(yTest, pred_svm))
print("Kappa = ", cohen_kappa_score(yTest, pred_svm))

Accuracy =  0.725
MCC =  0.0
Kappa =  0.0


In [25]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(xTest)
pred_prob

array([[0.65657657, 0.34342343],
       [0.68293164, 0.31706836],
       [0.671509  , 0.328491  ],
       [0.6574593 , 0.3425407 ],
       [0.64647062, 0.35352938],
       [0.66064176, 0.33935824],
       [0.71834273, 0.28165727],
       [0.6613974 , 0.3386026 ],
       [0.69022486, 0.30977514],
       [0.66590477, 0.33409523],
       [0.65383427, 0.34616573],
       [0.68790973, 0.31209027],
       [0.67778282, 0.32221718],
       [0.62691503, 0.37308497],
       [0.65926782, 0.34073218],
       [0.61959829, 0.38040171],
       [0.68709695, 0.31290305],
       [0.6802998 , 0.3197002 ],
       [0.68321658, 0.31678342],
       [0.68288127, 0.31711873],
       [0.67077531, 0.32922469],
       [0.705806  , 0.294194  ],
       [0.66445047, 0.33554953],
       [0.62945495, 0.37054505],
       [0.68330292, 0.31669708],
       [0.69705907, 0.30294093],
       [0.63771887, 0.36228113],
       [0.6699386 , 0.3300614 ],
       [0.69528181, 0.30471819],
       [0.64416209, 0.35583791],
       [0.

In [27]:
da = np.amax(pred_prob, axis=1) > threshold

In [29]:
print("Accuracy = ", accuracy_score(np.asarray(yTest)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yTest)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yTest)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  nan
MCC =  0.0
Kappa =  nan
Coverage =  0.0


/Users/timstassen/opt/anaconda3/envs/rdkit/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/timstassen/opt/anaconda3/envs/rdkit/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/timstassen/opt/anaconda3/envs/rdkit/lib/python3.11/site-packages/sklearn/metrics/_classification.py:673: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


Add the descriptors to the model

In [31]:
xNew = np.concatenate((x,allDescrsDf), axis=1)

In [38]:
xNtr, xNts, yNtr, yNts = train_test_split(xNew, y, test_size=0.20, random_state=seed)
scale = StandardScaler().fit(xNtr)
xNtr = scale.transform(xNtr)

In [39]:
# create grid search dictionary
param_grid = {"max_features": [xNtr.shape[1] // 10, xNtr.shape[1] // 7, xNtr.shape[1] // 5, xNtr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [40]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [41]:
# run model building
m.fit(xNtr, yNtr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [225, 322, 451, 752],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [42]:
m.best_score_

0.7074999999999999

In [43]:
# scale descriptors of the test set compounds
xNts = scale.transform(xNts)
# predict
pred = m.predict(xNts)
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [44]:
# calc statistics
print("Accuracy = ", accuracy_score(yNts, pred))
print("MCC = ", matthews_corrcoef(yNts, pred))
print("Kappa = ", cohen_kappa_score(yNts, pred))

Accuracy =  0.72
MCC =  0.09700731966572047
Kappa =  0.06588824020016681


In [45]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(xNts)
da = np.amax(pred_prob, axis=1) > threshold

print("Accuracy = ", accuracy_score(np.asarray(yNts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yNts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yNts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.8529411764705882
MCC =  0.0
Kappa =  0.0
Coverage =  0.34


In [46]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(xNtr, yNtr)

RandomForestClassifier(max_features=225, n_estimators=500, random_state=13)

In [47]:
imp = rf.feature_importances_
imp

array([1.61369133e-05, 3.29937160e-04, 4.91261696e-04, ...,
       4.75763530e-04, 9.72624933e-04, 2.28099621e-04])

In [48]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 2109 (0.019714)
2. feature 2052 (0.015663)
3. feature 2069 (0.014176)
4. feature 2071 (0.010537)
5. feature 2147 (0.008844)
6. feature 694 (0.008709)
7. feature 2067 (0.008686)
8. feature 2070 (0.008017)
9. feature 2085 (0.007987)
10. feature 2087 (0.007576)


features 1-2028 are different Morgan fingerprints